# Import dependencies

In [ ]:
import matlab.net.*
import matlab.net.http.*
import matlab.net.http.io.*

# Check API is available

In [ ]:
test_req = RequestMessage(matlab.net.http.RequestMethod.GET, [], []);
resp = test_req.send("http://host.docker.internal:8080/argo-toolbox/api/decoder/docs")
if(resp.StatusCode == 200)
    fprintf("✅ Decoder API available")
end

# Set file paths and WMONUM

In [ ]:
file_dir = "/home/jovyan/examples/data/6902892_raw"
float_info_dir = "/home/jovyan/examples/floats_meta/6902892"
wmonum = "6902892"

# Check if LFS example data has been pulled

In [ ]:
lfs_test_file = fullfile(file_dir, "co_20200629T083042Z_300234065895840_000004_000000_20420.txt")
fid = fopen(lfs_test_file, "r");
file_content = fscanf(fid, "%s");
fclose(fid)
if fid == -1
    fprintf('❌ File could not be opened. Check the path or permissions.\n');
else
    if contains(file_content, 'git-lfs')
        fprintf(['Example data files stored in Git LFS need to be pulled. ' ...
                 'Please install Git LFS and run `git lfs pull`. ' ...
                 'See JupyterLab documentation in the Argo Toolbox API repository for details.\n']);
    else
        fprintf('✅ Example data files found.\n');
    end
end

# Float configuration and metadata - Option 1: Construct manually

See the [Coriolis Decoder Manual](https://github.com/euroargodev/Coriolis-data-processing-chain-for-Argo-floats/tree/main/decArgo_doc/decoder_user_manual) for information on which parameters to include and how to populate this information.

In [ ]:
float_info = struct( ...
   "WMO", '6903014', ...
   "PTT", '300234068508780', ...
   "FLOAT_TYPE", 'PROVOR', ...
   "DECODER_VERSION", '5.45', ...
   "DECODER_ID", '212', ...
   "FRAME_LENGTH", '31', ...
   "CYCLE_LENGTH", '24', ...
   "DRIFT_SAMPLING_PERIOD", '1', ...
   "DELAI", '-1', ...
   "LAUNCH_DATE", '20001014060200', ...
   "LAUNCH_LON", '-4.626', ...
   "LAUNCH_LAT", '36.354', ...
   "END_DECODING_DATE", '20241109000000', ...
   "REFERENCE_DAY", '20201014', ...
   "DM_FLAG", '0' ...
);

meta_info = struct( ...
    "ARGO_USER_MANUAL_VERSION", '3.1', ...
    "PLATFORM_NUMBER", '6903014', ...
    "PTT", '850878', ...
    "IMEI", '300234068508780', ...
    "TRANS_SYSTEM", struct( ...
        "TRANS_SYSTEM_1", 'IRIDIUM' ...
    ) ...
    % add further metadata information
);

# Float configuration and metadata - Option 2: Load from file

In [ ]:
float_info = jsondecode(fileread(fullfile(float_info_dir, "float_info.json")));
meta_info  = jsondecode(fileread(fullfile(float_info_dir, "meta_info.json")));

# Decoder Configuration

In [ ]:
extra_args = struct( ...
    "DIR_OUTPUT_XML_FILE", "/mnt/data/output/xml/" ...
);

# Construct Request Metadata

In [ ]:
float_metadata = jsonencode(struct( ...
    "float_info", float_info, ...
    "float_meta_info", meta_info ...
));
configuration_override = jsonencode(extra_args);

# Open files and send request to decoder API

In [ ]:
url = "http://host.docker.internal:8080/argo-toolbox/api/decoder/decode_float/6903014";

txt_files = dir(fullfile(file_dir, "*.txt"));
file_paths = fullfile({txt_files.folder}, {txt_files.name});
files_provider = FileProvider(file_paths);

provider = MultipartFormProvider( ...
    "files", files_provider, ...
    "float_metadata", StringProvider(float_metadata), ...
    "configuration_override", StringProvider(configuration_override) ...
);

req = RequestMessage(matlab.net.http.RequestMethod.POST, [], provider);
resp = req.send(url);

fprintf("HTTP status: %s\n", string(resp.StatusCode));

# Download ZIP file generated by decoder

In [ ]:
output_dir = fullfile("/home/jovyan/examples/output", wmonum)
if resp.StatusCode == 200
    zipData = uint8(resp.Body.Data);
    fid = fopen(fullfile(output_dir, "output.zip"), "w");
    fwrite(fid, zipData, "uint8");
    fclose(fid);
    disp("ZIP file saved as output.zip");
else
    disp("Request failed:");
    disp(resp.Body.string);
end

# Extract ZIP file

In [ ]:
unzipDir = fullfile(fullfile("/home/jovyan/examples/output", wmonum, "unzipped"));
unzip(fullfile(output_dir, "output.zip"), unzipDir);

# Open dataset

In [ ]:
pattern = fullfile(unzipDir, 'tmp');
files = dir(fullfile(pattern, '**', wmonum, 'profiles', sprintf('R%s_001.nc', wmonum)));

if isempty(files)
    error('No matching NetCDF file found.');
else
    dataset_file = fullfile(files(1).folder, files(1).name);
end

ncinfo(dataset_file)

# Plot variables

In [ ]:
TEMP = ncread(dataset_file, 'TEMP');
if any(strcmp({ncinfo(dataset_file).Variables.Name}, 'PRES'))
    PRES = ncread(dataset_file, 'PRES');
elseif any(strcmp({ncinfo(dataset_file).Variables.Name}, 'DEPTH'))
    DEPTH = ncread(dataset_file, 'DEPTH');
end
figure;
imagesc(TEMP);    % simple image plot
colorbar;
title('TEMP');
xlabel('Profile #');
ylabel('Depth index');